In [ ]:
import os
import sys
from PIL import Image

def split_images(original_path, label_path, tile_size, output_path):
    # Open both images
    original = Image.open(original_path)
    label = Image.open(label_path)

    # Check that they have the same resolution
    if original.size != label.size:
        raise ValueError(f"Image sizes do not match: {img1.size} vs {img2.size}")

    width, height = original.size

    # Create output directories
    os.makedirs(os.path.join(output_path, "original"), exist_ok=True)
    os.makedirs(os.path.join(output_path, "label"), exist_ok=True)

    # Determine image mode (e.g., RGB, L, etc.)
    mode1 = original.mode
    mode2 = label.mode

    tile_count = 0
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            box = (x, y, min(x + tile_size, width), min(y + tile_size, height))
            tile_o = original.crop(box)
            tile_l = label.crop(box)

            # Create new black tiles and paste the cropped parts
            padded_tile_o = Image.new(mode1, (tile_size, tile_size), color=0)
            padded_tile_l = Image.new(mode2, (tile_size, tile_size), color=0)
            padded_tile_o.paste(tile_o, (0, 0))
            padded_tile_l.paste(tile_l, (0, 0))

            tile_name = f"tile_{tile_count:04d}.png"

            padded_tile_o.save(os.path.join(output_path, "original", tile_name))
            padded_tile_l.save(os.path.join(output_path, "label", tile_name))

            tile_count += 1
    
    print(f"Created {tile_count} tiles")


In [ ]:
BASE_DIR = "Museum labels"
TILE_SIZE = 1024

for name in os.listdir(BASE_DIR):
    path = os.path.join(BASE_DIR, name)
    if os.path.isdir(path):
        image1_path = os.path.join(path, f"{name}.JPG")
        image2_path = os.path.join(path, f"{name}_labels.png")
        os.makedirs(f"tiled_labels/{name}", exist_ok=True)
        split_images(image1_path, image2_path, TILE_SIZE, f"tiled_labels/{name}")
        print(f"Finished tiling {name}")